In [219]:
import requests,configparser
def get_cookie():
    config = configparser.ConfigParser()
    config.read('secrets.txt')
    cookie = config['session_info']['cookie']
    return cookie
def get_inputs(day):
    cookie, day = get_cookie(), int(day)
    headers = {'session': cookie}
    url = f'https://adventofcode.com/2022/day/{day}/input'
    session = requests.Session()
    resp = session.get(url,cookies=headers)
    return resp.text.split('\n')[:-1]

In [220]:
# DAY1
import numpy as np

def max_elf_snacks(data,top):
    elf_food = [0]
    for x in data:
        if x != '':
            elf_food[-1] += int(x)
        else:
            elf_food.append(0)
    top_elves = np.argpartition(elf_food,-top)[-top:]
    return sum(np.array(elf_food)[top_elves])

print(f'Part 1: {max_elf_snacks(get_inputs(1),1)}')
print(f'Part 2: {max_elf_snacks(get_inputs(1),3)}')


Part 1: 72602
Part 2: 207410


In [223]:
# DAY2
rps = {'A': [1,0,0],'B': [0,1,0], 'C': [0,0,1]}
rewards = np.array([[3,0,6], [6,3,0], [0,6,3]])

def rps_score(p,q):
    return np.matmul(np.matmul(p,rewards.T),q.T).trace() + sum(np.matmul([1,2,3],q.T))

def get_strategy(d, mode = 0):
    if mode == 0:
        # simple mode
        s = {'X': [1,0,0], 'Y': [0,1,0], 'Z': [0,0,1]}
    else:
        # part 2
        s = {'X': np.roll(rps[d[0]],-1),
            'Y': rps[d[0]],
            'Z': np.roll(rps[d[0]],+1)}
    return s[d[1]]
        
    
def get_moves(data, mode = 0):    
    p = np.array([rps[d[0]] for d in data])
    q = np.array([get_strategy(d,mode) for d in data])
    return p,q

def day2(d, mode = 0):
    data = [tuple(x.split(' ')) for x in d]
    return rps_score(*get_moves(data, mode))

print(f'Part 1: {day2(get_inputs(2),0)}')
print(f'Part 2: {day2(get_inputs(2),1)}')

Part 1: 13268
Part 2: 15508


In [222]:
# DAY3
def get_priority(item):
    return ord(item) - (96 if item.islower() else 64-26) 
        
def common_item(sack):
    h = int(len(sack)/2)
    c = set(sack[:h]).intersection(set(sack[h:]))
    for i in c:    
        return get_priority(i)

def common_item_3elf(sacks):
    c = set.intersection(*[set(sack) for sack in sacks])
    for i in c:    
        return get_priority(i)

def day3_p1():
    return sum(common_item(x) for x in get_inputs(3))

def day3_p2():
    data, n = get_inputs(3), 3
    chunk3 = [data[i:i+n] for i in range(0,len(data),n)]
    return sum(common_item_3elf(x) for x in chunk3)

print(f'Part 1: {day3_p1()}')
print(f'Part 2: {day3_p2()}')

Part 1: 7967
Part 2: 2716


In [268]:
# DAY4
import re
s = [[int(x) for x in re.findall('(\d+)',t)] for t in get_inputs(4)]
day4_p1 = sum(1 for x in s if (x[0]-x[2])*(x[1]-x[3])<=0)
day4_p2 = sum(1 for x in s if max(x)-min(x)<=x[3]-x[2]+x[1]-x[0])
print(f'Part 1: {day4_p1}')
print(f'Part 2: {day4_p2}')

Part 1: 424
Part 2: 804


In [345]:
# DAY5
def get_stacks(data):
    L = [[x for i,x in enumerate(line) if i%4==1] for line in d5[:d5.index('')-1]]
    L = (np.array(L).T).tolist()
    L = [l[::-1] for l in L]
    for l in L:
        while l[-1]==' ':
            l.pop()
    return L

def get_move_instructions(data):
    return [tuple(int(t) for t in re.findall('\d+',x)) for x in data if x.startswith('move') ]

def move_containers(stack,model,count,source,dest):
    x = [stack[source-1].pop() for i in range(count)]
    if model == 9000:
        stack[dest-1] += x
    else:
        stack[dest-1] += x[::-1]

def day5(model):
    d5 = get_inputs(5)
    stacks = get_stacks(d5)
    instructions = get_move_instructions(d5)
    for i in instructions:
        move_containers(stacks, model, *i)
    return ''.join([s.pop() for s in stacks])

print(f'Part 1: {day5(9000)}')
print(f'Part 2: {day5(9001)}')

Part 1: VCTFTJQCG
Part 2: GCFGLDNJZ
